# Trabalho final: Raciocinio Espacial Neuro-Simbólico com LTNtorch
**Professor:** Edjard Mota

**Disciplina:** Fundamentos de Inteligência Artificial (ICC260)

**Alunos:**
* **André Kaled Duarte Coutinho Andrade** (andre.andrade@icomp.ufam.edu.br)
* **Conceição Barbosa Rocha** (conceicao.rocha@icomp.ufam.edu.br)
* **Julia Evelyn Chaparro Ferreira** (julia.ferreira@icomp.ufam.edu.br)
* **Luiz Henrique Barbosa Costa** (luiz.costa@icomp.ufam.edu.br)
* **Marcus Phablo Pereira de Oliveira** (marcus.oliveira@icomp.ufam.edu.br)
* **Paulo Victor Fernandes de Melo** (paulo.fernandes@icomp.ufam.edu.br)
* **Sven Maximilian Kalisch** (sven.kalisch@icomp.ufam.edu.br)
* 
## Resumo do Projeto
Este projeto implementa um agente neuro-simbólico utilizando Logic Tensor Networks (LTN) para raciocinar sobre um ambiente simplificado. O objetivo é aprender e validar relações espaciais (esquerda, direita, acima, abaixo, entre) e atributos de objetos (forma, cor, tamanho) baseando-se em vetores de características, não em imagens brutas.

## Instalação e configuração inicial

In [3]:
!pip install ltn torch matplotlib numpy

import torch
import ltn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.path as mpath

torch.manual_seed(42)

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


ModuleNotFoundError: No module named 'torch'

## Geração de dados e plotagem


In [4]:
def get_custom_markers():
    """
    Cria caminhos (Paths) vetoriais para desenhar Cone e Cilindro
    com aparência 3D usando curvas de Bézier.
    """
    # Gemini que fez
    # --- CONE (Triângulo + Base Curva) ---
    verts_cone = [
        (0.0, 1.0),
        (-0.5, -1.0),
        (-0.5, -1.35),
        (0.5, -1.35),
        (0.5, -1.0),
        (0.0, 1.0)
    ]
    codes_cone = [
        mpath.Path.MOVETO,
        mpath.Path.LINETO,
        mpath.Path.CURVE4,
        mpath.Path.CURVE4,
        mpath.Path.CURVE4,
        mpath.Path.LINETO
    ]
    path_cone = mpath.Path(verts_cone, codes_cone)

    # --- CILINDRO (Retângulo + Topo/Base Curvos) ---
    verts_cyl = [
        (-0.5, 1.0),  # Topo Esq
        (-0.5, -1.0), # Baixo Esq
        (0.5, -1.0),  # Baixo Dir (Curva suave entre estes dois)
        (0.5, 1.0),   # Topo Dir
        (-0.5, 1.0),  # Volta Topo Esq (Curva suave entre estes dois)
        (-0.5, 0.95), # Um pequeno detalhe para fechar o "loop" visualmente
    ]
    codes_cyl = [
        mpath.Path.MOVETO,
        mpath.Path.LINETO,
        mpath.Path.CURVE4, # Controle 1 da base
        mpath.Path.CURVE4, # Controle 2 da base
        mpath.Path.CURVE4, # Fim da base
        mpath.Path.LINETO,
        mpath.Path.CURVE4, # Controle 1 do topo
        mpath.Path.CURVE4, # Controle 2 do topo
        mpath.Path.CURVE4  # Fim do topo
    ]
    
    verts_cyl_fixed = [
        (-0.5, 1.0),  # Start Top-Left
        (-0.5, -1.0), # Line to Bottom-Left
        
        # Curva de baixo (precisa de 2 pontos de controle + 1 final)
        (-0.5, -1.4), # Controle 1
        (0.5, -1.4),  # Controle 2
        (0.5, -1.0),  # Ponto Final (Bottom-Right)
        
        (0.5, 1.0),   # Line to Top-Right
        
        # Curva de cima (para fechar a tampa)
        (0.5, 1.4),   # Controle 1
        (-0.5, 1.4),  # Controle 2
        (-0.5, 1.0)   # Ponto Final (Top-Left)
    ]
    
    path_cyl = mpath.Path(verts_cyl_fixed, codes_cyl)
    
    return path_cone, path_cyl

def get_dataset_simplificado(n_exemplos=25):
    """
    Gera dataset CLEVR simplificado (x, y, cores, formas, tamanho).
    Vetor [11 features]:
    [0,1]: x, y
    [2,3,4]: r, g, b
    [5,6,7,8,9]: circle, square, cylinder, cone, triangle
    [10]: size (0=pequeno, 1=grande)
    """
    data = []
    labels = []
    
    nome_formas = ["Circle", "Square", "Cylinder", "Cone", "Triangle"]
    nome_cores = ["Red", "Green", "Blue"]
    nome_tamanhos  = ["Small", "Large"]
    
    for _ in range(n_exemplos):
        # posição
        x, y = np.random.rand(), np.random.rand()
        
        # cor (one-hot)
        cor_idx = np.random.randint(0, 3)
        cor_vet = [0.0]*3
        cor_vet[cor_idx] = 1.0
        
        # forma (one-hot)
        forma_idx = np.random.randint(0, 5)
        forma_vet = [0.0]*5
        forma_vet[forma_idx] = 1.0
        
        # tamanho
        grande = np.random.rand() > 0.5
        tamanho_val = 1.0 if grande else 0.0
        
        vet = [x, y] + cor_vet + forma_vet + [tamanho_val]
        data.append(vet)
        
        desc = f"{nome_tamanhos[int(tamanho_val)]} {nome_cores[cor_idx]} {nome_formas[forma_idx]}"
        labels.append(desc)
        
    return torch.tensor(data, dtype=torch.float32), labels

def plot_dataset_simplificado(data, labels, title="Cenário CLEVR Gerado"):
    plt.figure(figsize=(8, 8))
    plt.xlim(-0.05, 1.05)
    plt.ylim(-0.05, 1.05)
    plt.grid(True, linestyle='--', alpha=0.5)
    marcador_cone, marcador_cilindro = get_custom_markers()
    
    # 0: circle, 1: square, 2: cylinder, 3:cone, 4:triangle
    marcadores = ['o', 's', marcador_cilindro, marcador_cone, '^']
    
    # 0:Red, 1:Green, 2:Blue
    cores = ['red', 'green', 'blue']
    
    for i in range(len(data)):
        obj = data[i]
        x, y = obj[0].item(), obj[1].item()
        
        # Identificar cor (argmax dos indices 2,3,4)
        c_idx = torch.argmax(obj[2:5]).item()
        c = cores[c_idx]
        
        # Identificar forma (argmax dos indices 5,6,7,8,9)
        s_idx = torch.argmax(obj[5:10]).item()
        m = marcadores[s_idx]
        
        # tamanho (indice 10)
        size_val = obj[10].item()
        # grande: area=300, pequeno: area=100
        base_size = 500 if size_val > 0.5 else 200

        # cilindro ou cone
        if s_idx == 2 or s_idx == 3:
            base_size *= 1.8
        
        plt.scatter(x, y, c=c, marker=m, s=base_size, edgecolors='black', label=labels[i], alpha=0.8, linewidths=1.5)
        offset = 0.035 if s_idx in [2,3] else 0.025
        plt.text(x, y+offset, str(i), fontsize=10, ha='center', fontweight='bold', color='black')

    plt.title(title)
    plt.xlabel("X (posição)")
    plt.ylabel("Y (posição)")
    plt.tight_layout()
    plt.show()

# testando
data_batch, text_labels = get_dataset_simplificado(25)
plot_dataset_simplificado(data_batch, text_labels)

NameError: name 'np' is not defined